In [2]:
!pip install nltk numpy pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 4.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/12.6 MB 4.2 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/12.6 MB 5.0 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.6 MB 6.5 MB/s eta 0:00:02
   ------------------------ --------------- 7.9/12.6 MB 7.6 MB/s eta 0:00:01
   ------------------------------ --------- 9.7/12.6 MB 8.0 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.6 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------- ------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import *
import string


nltk.download('stopwords')

# Define stopword set
stopword = set(stopwords.words('english'))

print("Setup complete! Stopwords loaded:", list(stopword)[:10])  # Display first 10 stopwords for confirmation


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\B-Panda\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Setup complete! Stopwords loaded: ['a', 'have', 'had', 'nor', 'again', 'himself', 'with', 'our', "aren't", 'most']


In [ ]:
df = pd.read_csv("Data/labeled_data.csv")
lemmatizer = WordNetLemmatizer()


nltk.download('all')

In [4]:
def cleanText(text):
    text = str(text).lower()
    text = re.sub('', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split() if word not in stopword]
    text = " ".join(text)
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    text = " ".join(text)
    return text


In [5]:
df["tweet"] = df["tweet"].apply(cleanText)

for i , item in enumerate(df["hate_speech"]):
    if item != 0 and item != 1 :
        df["hate_speech"][i] = 1

C:\Users\zouha\AppData\Local\Temp\ipykernel_17232\3548468260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hate_speech"][i] = 1
C:\Users\zouha\AppData\Local\Temp\ipykernel_17232\3548468260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hate_speech"][i] = 1
C:\Users\zouha\AppData\Local\Temp\ipykernel_17232\3548468260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hate_speech"][i] = 1
C:\Users\zouha\AppData\Lo

In [6]:
df.head(100)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely woman shouldnt complain cleani...
1,1,3,0,3,0,1,rt boy dat coldtyga dwn bad cuffin dat hoe place
2,2,3,0,3,0,1,rt urkindofbrand dawg rt ever fuck bitch start...
3,3,3,0,2,1,1,rt cganderson vivabased look like tranny
4,4,6,0,6,0,1,rt shenikaroberts shit hear might true might f...
...,...,...,...,...,...,...,...
95,96,3,0,3,0,1,causewereguys going back school suck dick hoe ...
96,97,3,0,3,0,1,causewereguys way fuck yo bitch year old
97,98,3,0,3,0,1,celeynichole whitethunduh come never bring foo...
98,99,3,0,3,0,1,chadmfverbeck richnow doesnt show hella tinder...


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


x = np.array(df["tweet"])
y= np.array(df['hate_speech'])

cv = CountVectorizer()
x = cv.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , confusion_matrix

logreg = LogisticRegression(max_iter=10000)
logreg.fit(x_train, y_train)
logreg_predict = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_predict, y_test)
print(logreg_acc)
print(f"Accuracy: {logreg_acc}")
logreg_confusion = confusion_matrix(y_test, logreg_predict)
# Create a DataFrame for the confusion matrix for better readability
conf_matrix_df = pd.DataFrame(logreg_confusion, 
                              index=[f"Actual {i}" for i in range(len(logreg_confusion))], 
                              columns=[f"Predicted {i}" for i in range(len(logreg_confusion))])

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix_df)

0.8297357272543877
Accuracy: 0.8297357272543877
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0         3750          217
Actual 1          627          363


In [15]:
text = "i hate white people"
text = cleanText(text)
print(text)
text = cv.transform([text])
logreg.predict(text)[0]

hate white people


1

In [10]:
import joblib

joblib.dump(logreg, 'regression_model.joblib')
joblib.dump(cv, 'vectoriser.joblib')

['vectoriser.joblib']